In [1]:
import pandas as pd
import json
import yfinance as yf
from datetime import datetime, timedelta
from backend import base, reporting
from utils import utils
import plotly.express as px

In [2]:
START_DT = '2022-01-10'
now_ = utils.date2str(datetime.now())

In [3]:
db_connector = base.BaseDBConnector('core.db')
data_fetcher = base.DataFetcher(db_connector)
reporter = reporting.Reporter(data_fetcher)

In [8]:
# FETCH MISSING DATA
missing_data_getter = base.DBUpdater(db_conn=db_connector)
_ = missing_data_getter.fetch_missing_fx(START_DT, now_)
_ = missing_data_getter.fetch_missing_securities_yf(START_DT, now_)

    TICKER FETCH_START_DT FETCH_END_DT
0     AAPL     2022-03-18   2022-03-18
1     AMZN     2022-03-18   2022-03-18
2  ETH-USD     2022-03-18   2022-03-18
3     SPGI     2022-03-18   2022-03-18
4   SPY5.L     2022-03-18   2022-03-18
5     TSLA     2022-03-18   2022-03-18
UNIQUE constraint failed: SECURITY_VALUES.TICKER, SECURITY_VALUES.DATE


In [9]:
# GET PORTFOLIO EVOLUTION FROM START_DT
ref_cost = reporter.get_portfolio_info(1, now_)['cost']
df_evolution = pd.DataFrame()
df_evolution['DT'] = list(utils.daterange(START_DT, now_))
df_evolution['RETURN'] = df_evolution['DT'].apply(lambda x: reporter.get_portfolio_info(1, x, ref_cost=ref_cost)['profit_perc'])
px.line(df_evolution, x='DT', y=['RETURN'], title='Portfolio ROI')


In [10]:
import numpy as np

df_composition = reporter.get_portfolio_info(1, now_, ref_cost=ref_cost)['df']
df_composition['PROFIT'] = np.round(df_composition['VALUE'] - df_composition['TOTAL_COST'], 2)
df_composition['PROFIT%'] = np.round(df_composition['PROFIT'] * 100 / df_composition['TOTAL_COST'], 2)
df_composition

,TICKER,N_SHARES,TOTAL_COST,DT,PRICE,FX,VALUE,PROFIT,PROFIT%
0,AAPL,7.83,4584.324060,2022-03-18,160.619995,4.4807,5635.172815,1050.85,22.92
1,AMZN,0.07,926.220792,2022-03-18,3144.780029,4.4807,986.357115,60.14,6.49
2,ETH-USD,0.32,5141.001600,2022-03-18,2814.854492,4.4807,4036.005942,-1105.00,-21.49
3,FP,6068.00,11365.640000,2022-03-18,1.960000,1.0000,11893.280000,527.64,4.64
4,SNN,72.00,3258.990000,2022-03-18,44.950000,1.0000,3236.400000,-22.59,-0.69
5,SPGI,1.28,2004.654080,2022-03-18,402.660004,4.4807,2309.374317,304.72,15.20
6,SPY5.L,6.00,11996.128000,2022-03-18,438.450012,4.4807,11787.377860,-208.75,-1.74
7,TSLA,0.19,573.158712,2022-03-18,871.599976,4.4807,742.021825,168.86,29.46
8,TVBETETF,1042.00,17265.940000,2022-03-18,17.782000,1.0000,18528.844000,1262.90,7.31


In [11]:
fig_cost_pie = px.pie(df_composition, names='TICKER', values='TOTAL_COST', title='Cost distribution', width=500)
fig_value_pie = px.pie(df_composition, names='TICKER', values='VALUE', title = 'Value distribution', width=500)
fig_value_pie